In [47]:
import requests
import requests.auth
import datetime as dt

import pandas as pd

from psaw import PushshiftAPI
api = PushshiftAPI()

import pprint
pp = pprint.PrettyPrinter(indent=4)

#natural language processing libraries
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')
import time
from textblob import Blobber
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
tb = Blobber(analyzer=NaiveBayesAnalyzer())

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\alex\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alex\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Pushshift API wrapper query
    * automatic rate limiting
    * faster queries as it doesn't actually directly query reddit's servers
    * better searching

### Pushshift submission API request

In [3]:
# get all threads from the bitcoinmarket subreddit, gen is a generator 

start_epoch=int(dt.datetime(2013, 1, 1).timestamp())
gen = api.search_submissions(after=start_epoch,
        subreddit='bitcoinmarkets', #specify the subforum
        sort = 'asc',
            # filter=['url','author', 'title', 'subreddit'], # choose which dictionary keys to display
        limit=100000)

In [4]:
# iterate through the generator to create a submission array
# x.d_ is a method that converts each submission object to a dictionary
results = list(map(lambda x: x.d_, gen))

In [5]:
df = pd.DataFrame(results)
df.to_csv('bitcionmarkets_threads.csv')
df.to_json('')

### Pushshift comment API request

In [14]:
# searching for comments in the bitcoinmarkets subredidt

def query_comments(search):
    """ Finds comments within a particular subreddit containing a phrase
    
    Keyword arguments:
    search (string) -- phrase to search for

    """
    gen = api.search_comments(q=search, subreddit='bitcoinmarkets')
    max_response_cache = 100000
    cache = []

    for c in gen:
        cache.append(c.d_)

        # Omit this test to actually return all results
        if len(cache) >= max_response_cache:
            break
            
    return cache

In [15]:
bullish = query_comments("bullish")
bearish = query_comments("bearish")
print(len(bullish))
print(len(bearish))

25389
15792


In [25]:
def get_sentiment(comment):
    """ Returns texblob sentiment analysis for a comment's body
    
    Keyword arguments:
    comment (dict) -- element from a query_comments array
    """
    return TextBlob(comment['body']).sentiment

In [58]:
from multiprocessing.dummy import Pool as ThreadPool
pool = ThreadPool()
# utalize all cores
bullish_sentiments = pool.map(lambda x: tb(x['body']).sentiment, bullish)
#close the pool and wait for the work to finish
pool.close()
pool.join()

In [62]:
bullish_polarity = list(map(lambda x: x.p_pos, bullish_sentiments))

In [69]:
import numpy as np
print(bullish[np.argmin(bullish_polarity)]['body'])

I'm insanely bullish about the price in the medium term and I'd love to leverage those gains up. But im a bit thick and even if I wasnt I dont think low leverage is a good idea at the moment. 60/40 seems crazy irresponsible. But I can get a lot of Yolos for that price even with randomly selected my way in I feel like I'm much more likely to come right. One random shot every week for the next few months at say 50 times seems very likely to come right at least once, and maybe right a lot of times. Of course the price actually needs to rise, but if it drops for months low leverage is dead anyway. I'm making excuses for greed, but I wonder if there is not a way to make this about as safe as holding alone. Say 10 percent fiat ashedge/rebut at the bottom. And a random leveraged punt every week for a year. So much

Upside, and the odds seem good? I'm a bit clueless though. Can still get stopped out I guess. What are the odds of Getting unlucky is the price is rising at a good rate? If I was d

In [60]:
bullish_sentiments[0]

Sentiment(classification='pos', p_pos=0.7249996645149416, p_neg=0.27500033548505376)

In [67]:
dt.datetime.fromtimestamp(bearish[-1]['created_utc']).strftime("%m/%d/%Y, %H:%M:%S")

'04/21/2013, 12:01:00'